<a href="https://colab.research.google.com/github/racalado/Visao_Computacional_Dados/blob/main/Visao_Computacional_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title CONTAGEM DADOS - 21

import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_image(title, img, cmap=None):
    plt.figure(figsize=(6, 6))
    plt.title(title)
    plt.axis("off")
    plt.imshow(img, cmap=cmap)
    plt.show()

def count_dice_points(image_path):
    # Ler a imagem
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    show_image("Imagem Original", image_rgb)

    # Converter para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    show_image("Escala de Cinza", gray, cmap='gray')

    # Aplicar limiarização adaptativa
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    show_image("Limiarização Adaptativa", thresh, cmap='gray')

    # Encontrar contornos dos dados
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_contours = image_rgb.copy()
    cv2.drawContours(image_contours, contours, -1, (0, 255, 0), 2)
    show_image("Contornos Identificados", image_contours)

    # Lista para armazenar os pontos de cada dado
    dice_points = []

    # Processar cada contorno (dado)
    for contour in contours:
        # Filtrar contornos muito pequenos ou muito grandes
        area = cv2.contourArea(contour)
        if 1000 < area < 50000:  # Ajuste conforme necessário
            # Criar máscara para o dado atual
            mask = np.zeros(gray.shape, np.uint8)
            cv2.drawContours(mask, [contour], 0, 255, -1)
            show_image("Máscara do Dado", mask, cmap='gray')

            # Encontrar pontos no dado
            points_mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            show_image("Pontos Destacados", points_mask, cmap='gray')

            # Encontrar blobs (pontos)
            params = cv2.SimpleBlobDetector_Params()
            params.filterByArea = True
            params.minArea = 10
            #params.maxArea = 500
            params.filterByCircularity = True
            params.minCircularity = 0.8

            detector = cv2.SimpleBlobDetector_create(params)
            keypoints = detector.detect(points_mask)

            # Desenhar keypoints na imagem
            image_keypoints = cv2.drawKeypoints(image_rgb, keypoints, np.array([]), (0, 0, 255),
                                                cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            show_image("Detecção dos Pontos", image_keypoints)

            dice_points.append(len(keypoints))

    return dice_points

# Caminho para a imagem
image_path = '/content/red_dices.jpg'

# Contar pontos
points = count_dice_points(image_path)

# Exibir pontos e soma total
print("Pontos em cada dado:", points)
print("Soma total dos pontos:", sum(points))